# Testing WDRC With Mixed SIgnal C++ and Matlab

## Summary - Near Perfect Match

As close as can be with c++ using 32-bit floats.

In [ ]:
# make Jupyter use the whole width of the browser
from IPython.display import Image, display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import numpy as np
from mlab import call_matlab, generate_sine_waves, plot_fft
from rtmha.elevenband import Wdrc11
import plotly.graph_objects as go

In [ ]:
# interactive must be False to create PDF output
interactive = False

def plot_response(title, output, B1, B2, AT=None, RT=None):
    fig = go.Figure()

    # Create x values. 32 samples per millisecond
    x = np.array(range(len(output)))/32.0

    fig.add_trace(go.Scatter(x=x, 
                y=output, 
                opacity=.5))

    fig.update_layout(title=title,
                    xaxis_title='Time(ms)',
                    yaxis_title='Output',
                    template='plotly_white')
    fig.add_hline(y=B1, line_dash="dash", line_color="black", opacity=.5, annotation_text="Desired Level")
    fig.add_hline(y=B2, line_dash="dash", line_color="black", opacity=.5, annotation_text="Desired Level")
    if AT:
        fig.add_vline(x=AT, line_dash="solid", line_color="green", opacity=.5)
    if RT:
        fig.add_vline(x=RT, line_dash="solid", line_color="green", opacity=.5)
    if interactive:
        fig.show()        
    else:
        fig.write_image("temp.png", engine="kaleido")
        display(Image("temp.png", width=4096))

def plot_response2(title, out1, lab1, out2, lab2, B1=None, B2=None, AT=None, RT=None):
    fig = go.Figure()

    # Create x values. 32 samples per millisecond
    x = np.array(range(len(out1)))/32.0

    fig.add_trace(go.Scatter(x=x, 
                y=out1,
                name=lab1,
                opacity=.5))
    fig.add_trace(go.Scatter(x=x, 
                y=out2,
                name=lab2,
                opacity=.5))
    fig.update_layout(title=title,
                    xaxis_title='Time(ms)',
                    yaxis_title='Output',
                    template='plotly_white')
    if B1:
        fig.add_hline(y=B1, line_dash="dash", line_color="black", opacity=.5, annotation_text="Desired Level")
    if B2:
        fig.add_hline(y=B2, line_dash="dash", line_color="black", opacity=.5, annotation_text="Desired Level")
    if AT:
        fig.add_vline(x=AT, line_dash="solid", line_color="green", opacity=.5)
    if RT:
        fig.add_vline(x=RT, line_dash="solid", line_color="green", opacity=.5)
    if interactive:
        fig.show()        
    else:
        fig.write_image("temp.png", engine="kaleido")
        display(Image("temp.png", width=4096))


In [ ]:
g50 = np.ones(11) * 30
g80 = np.ones(11) * 10
kneelow = np.ones(11) * 45
band_mpo = np.ones(11) * 100
AT = np.ones(11) * 10
RT = np.ones(11) * 100

In [ ]:
min_phase=1
align=1

# Mixed signal for 200ms at 32K sampling rate
_, inp = generate_sine_waves([200, 1000, 5000], .2)

# set to 55dB except the interval [50-100]ms
inp[0:1600] *= 10**(55/20)
inp[1600:3200] *= 10**(90/20)
inp[3200:-1] *= 10**(55/20)

In [ ]:
from scipy.signal import hilbert
analytic_signal = hilbert(inp)
amp = np.abs(analytic_signal)

# check input signal 
plot_response('Input (55db except for 90db for 50ms)', 20*np.log10(amp), 55, 90)

In [ ]:
w = Wdrc11(g50, g80, kneelow, band_mpo, AT, RT, len(inp), min_phase, align)
out_all = w.wdrc(inp)

In [ ]:
# Compare with Matlab
res = call_matlab(min_phase, align, inp, g50, g80, kneelow, band_mpo, AT, RT)

In [ ]:
# check all the bands
for i in range(11):
    analytic_signal = hilbert(res['output'][i])
    m = np.abs(analytic_signal)
    analytic_signal = hilbert(w.get_band(i))
    c = np.abs(analytic_signal)
    plot_response2(f'Band {i}', 20*np.log10(m), 'Matlab', 20*np.log10(c), 'C++', w.B1[10], w.B2[10], 60, 200)

In [ ]:
all_mlab = res['output'][0]
for i in range(1,11):
    all_mlab += res['output'][i]
    
analytic_signal = hilbert(all_mlab)
m = np.abs(analytic_signal)
analytic_signal = hilbert(out_all)
c = np.abs(analytic_signal)
plot_response2(f'All Bands', 20*np.log10(m), 'Matlab', 20*np.log10(c), 'C++', w.B1[10], w.B2[10], 60, 200)